In [ ]:
import os
from pprint import pprint

from osgeo import ogr  # vector
from osgeo import osr  # spatial reference

## Shell commands (batch commands)
- ogrinfo can be combined with SQL syntax to select features from a dataset
- ogr2ogr (translations, conversion, reprojection, etc.) can also be used with SQL syntax

In [ ]:
!pwd

In [ ]:
# Supported formats
!ogrinfo --formats

In [ ]:
# Inspect shapefile metadata
!ogrinfo -so /opt/diml-service/data/us_states tl_2014_us_state

In [ ]:
# Convert shapefile to geojson
#ogr2ogr -f "GeoJSON" "[output_path]" "[datasource path]"

# Merge multiple vector files (shapefiles)
#ogrmerge.py in GDAL/scripts folder

### SQL Server shapefile export

In [ ]:
%%bash

# Connect to MSSQL
ogrinfo -so \
    "MSSQL:Driver={ODBC Driver 17 for SQL Server};Server=[FQDN];Database=;Uid=;Pwd=" \
    landgrid.State

In [ ]:
%%time
%%bash

# Export from MSSQL to Shapefile
ogr2ogr -overwrite -a_srs "EPSG:4326" -f "ESRI Shapefile" \
    /opt/diml-service/data/mssql_test \
    "MSSQL:Driver={ODBC Driver 17 for SQL Server};Server=[FQDN];Database=;Uid=;Pwd=" \
    landgrid.OhSection

### PostGis Operations

In [ ]:
%%time
%%bash

# Inspect table layer
ogrinfo -so \
    "PG:dbname=db host=host port=5432 user=sa password=pw" \
    maps.tx_ector

In [ ]:
%%time
%%bash

# Write table layer to shapefile
ogr2ogr -overwrite -a_srs "EPSG:4269" -f "ESRI Shapefile" \
    /opt/diml-service \
    "PG:dbname=db host=host port=5432 user=sa password=pw" \
    maps.tx_ector

In [ ]:
%%time
%%bash

# Inspect shapefile
ogrinfo -so /opt/diml-service/maps.tx_ector.shp maps.tx_ector

In [ ]:
%%time
%%bash

# Write shapefile to table
ogr2ogr -update -append -f "PostgreSQL" --config PG_USE_COPY YES -nlt PROMOTE_TO_MULTI \
    "PG:dbname=ScratchDB host=postgres-gis port=5432 user=sa password=HelloWorld1" \
    /opt/diml-service/maps.tx_ector.shp \
    -nln maps.tx_ector

## Open Shapefile

In [ ]:
shp_path = os.path.join(os.environ["DATA_DIR"], 'us_states', 'tl_2014_us_state.shp')

driver = ogr.GetDriverByName("ESRI Shapefile")
dataSource = driver.Open(shp_path, 0)
layer = dataSource.GetLayer(0)

print(layer.GetLayerDefn())
print(f"Spatial Ref: {layer.GetSpatialRef()}")
print(f"#Features: {layer.GetFeatureCount()}")

# Enumerate features attributes and geometry
for feature in layer:
    pprint(feature)
    print(feature.GetField("NAME"))
    
    geom = feature.GetGeometryRef()
    print(geom.Centroid().ExportToWkt())
    
# for i in range(layer.GetFeatureCount()):
#     print(layer.GetFeature(i).GetField("NAME"))
    


## Create Polygon

In [ ]:
r = ogr.Geometry(ogr.wkbLinearRing)
r.AddPoint(1,1)
r.AddPoint(5,1)
r.AddPoint(5,5)
r.AddPoint(1,5)
r.AddPoint(1,1)
poly = ogr.Geometry(ogr.wkbPolygon)
poly.AddGeometry(r)

print(poly.ExportToWkt())

## Create Polygon from GeoJSON

In [ ]:
geojson = """
{"type":"Polygon","coordinates":[[[1,1],[5,1],[5,5],[1,5], [1,1]]]}
"""
polygon = ogr.CreateGeometryFromJson(geojson)
print(polygon) 

## Geometric Operations

In [ ]:
print(polygon.Centroid())
print(polygon.GetBoundary())
print(polygon.ConvexHull())
print(polygon.Buffer(0))

point = ogr.Geometry(ogr.wkbPoint)
point.AddPoint(10, 10)
polygon.Contains(point)

## Create Shapefile

In [ ]:
# 1 set the spatial reference (WGS84)
spatial_ref = osr.SpatialReference()
spatial_ref.SetWellKnownGeogCS('WGS84')

# 2 create a new shapefile
shp_path = os.path.join(os.environ["DATA_DIR"], 'shapefile_out', 'ogr_ex.shp')
driver = ogr.GetDriverByName('ESRI Shapefile')
shape_data = driver.CreateDataSource(shp_path)

# 3 create the layer
new_layer = shape_data.CreateLayer('polygon_layer', spatial_ref, ogr.wkbPolygon)

id_field = ogr.FieldDefn("ID", ogr.OFTInteger)
id_field.SetWidth(4)
new_layer.CreateField(id_field)
    
# 4 geometry is put inside feature
feature = ogr.Feature(new_layer.GetLayerDefn())
feature.SetFID(0)
feature.SetField("ID", 21)

geojson = '{"type":"Polygon","coordinates":[[[1,1],[5,1],[5,5],[1,5], [1,1]]]}'
polygon = ogr.CreateGeometryFromJson(geojson)

feature.SetGeometry(polygon)

# 5 feature is put into layer
new_layer.CreateFeature(feature)

# THIS IS ESSENTIAL - basically closes the file handle
#feature.Destroy()

# THIS IS ESSENTIAL - basically closes the file handle
shape_data.FlushCache()
shape_data.Destroy()  # or shape_data = None


In [ ]:
# NOTE: DBF_DATE_LAST_UPDATE exists. Could be used for CT?
!ogrinfo /opt/diml-service/data/shapefile_out/ogr_ex.shp ogr_ex

## Use a Spatial Filter

In [ ]:
shp_path = os.path.join(os.environ["DATA_DIR"], 'us_states', 'tl_2014_us_state.shp')
driver = ogr.GetDriverByName('ESRI Shapefile')
data_source = driver.Open(shp_path, 0)  # read only

layer = data_source.GetLayer()

# Pass in the coordinates for the data frame to the SetSpatialFilterRect() function. 
# This filter creates a rectangular extent and selects the features inside the extent
layer.SetSpatialFilterRect(-102, 26, -94, 36)

for feature in layer:
    print(feature.GetField("NAME"))
    
data_source.Destroy()